In [1]:
# https://data.gov.sg/dataset/resale-flat-prices

In [2]:
import pandas as pd
import numpy as np
import requests

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
%cd /content/gdrive/MyDrive/6242_PROJECT/data

/content/gdrive/MyDrive/6242_PROJECT/data


In [5]:
%ls

 amenities.csv
 bus_stops.csv
 carparks.csv
 data-download.png
 data_hawker.csv
 data_hawker.gsheet
 data_malls.csv
 data_MRT.csv
 data_prischools.csv
 data_prischools.gsheet
'eating_establishments (1).csv'
 eating_establishments.csv
 eating-establishments.kml
 general-information-of-schools.csv
 hdb.csv
 listing-of-supermarkets.csv
 parks.csv
 parks-kml.kml
 processed/
 resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv
 secondary_schools.csv
 secondary_schools.gsheet
 supermarkets.csv
 taxi_stands.csv
 ura.csv
 ura_join_bus_stops.csv


In [6]:
hdb = pd.read_csv("resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv")

In [7]:
hdb

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0
...,...,...,...,...,...,...,...,...,...,...,...
111928,2021-11,YISHUN,5 ROOM,864,YISHUN AVE 4,04 TO 06,122.0,Improved,1988,65 years 08 months,603000.0
111929,2021-11,YISHUN,5 ROOM,315B,YISHUN AVE 9,07 TO 09,112.0,Improved,2015,93 years,605000.0
111930,2021-11,YISHUN,5 ROOM,505D,YISHUN ST 51,13 TO 15,112.0,Improved,2016,93 years 05 months,628000.0
111931,2021-11,YISHUN,EXECUTIVE,352,YISHUN RING RD,01 TO 03,146.0,Maisonette,1988,65 years 10 months,650000.0


In [8]:
hdb["block"]=  hdb["block"] + " "+ hdb["street_name"] 

In [9]:
hdb["block"]

0         406 ANG MO KIO AVE 10
1          108 ANG MO KIO AVE 4
2          602 ANG MO KIO AVE 5
3         465 ANG MO KIO AVE 10
4          601 ANG MO KIO AVE 5
                  ...          
111928         864 YISHUN AVE 4
111929        315B YISHUN AVE 9
111930        505D YISHUN ST 51
111931       352 YISHUN RING RD
111932         837 YISHUN ST 81
Name: block, Length: 111933, dtype: object

In [10]:
test = requests.get("https://developers.onemap.sg/commonapi/search?searchVal={}&returnGeom=Y&getAddrDetails=Y&pageNum=1".format("406 ANG MO KIO AVE 10"))
print(test.json()['results'])

[{'SEARCHVAL': '406 ANG MO KIO AVENUE 10 SINGAPORE 560406', 'BLK_NO': '406', 'ROAD_NAME': 'ANG MO KIO AVENUE 10', 'BUILDING': 'NIL', 'ADDRESS': '406 ANG MO KIO AVENUE 10 SINGAPORE 560406', 'POSTAL': '560406', 'X': '30288.2346631354', 'Y': '38229.0674628187', 'LATITUDE': '1.36200453938712', 'LONGITUDE': '103.853879910407', 'LONGTITUDE': '103.853879910407'}]


In [11]:
def getLatLong(x):
  address = x["block"]
  matches = requests.get("https://developers.onemap.sg/commonapi/search?searchVal={}&returnGeom=Y&getAddrDetails=Y&pageNum=1".format(address)).json()['results']
  try:
    first_match = matches[0]
    lat,long = first_match['LATITUDE'],first_match['LONGITUDE']
    isLatLongAvailable='True'
    return (lat,long,isLatLongAvailable)
  except IndexError:
    isLatLongAvailable='False'
    return ("unknown","unknown",isLatLongAvailable)

In [12]:
# hdb=hdb.iloc[:5,:]

In [ ]:
hdb_with_lat_long=pd.concat([hdb,hdb.apply(getLatLong,axis=1,result_type = "expand")],axis=1) 

In [ ]:
hdb_with_lat_long = hdb_with_lat_long.rename(columns={0:"lat",1:"long",2:"isLatLongAvailable"})

In [ ]:
hdb_with_lat_long

In [ ]:
hdb_lat_long.to_csv("./hdb.csv")

DON'T RUN ANYTHING BEYOND THIS